Project developed by João Pedro Cunha on April 2024

This project involves using particle-resolved direct numerical simulations (PR-DNS) to study the flow past a randomly arranged array of cylinders. The objective is to explore how particle interactions influence individual drag forces at a steady state.

PR-DNS offers highly accurate solutions to the Navier-Stokes equations at a microscale level, although it demands significant computational resources. Key assumptions include treating the fluid as incompressible, isothermal, and Newtonian, along with applying triple periodic boundary conditions. Cylinder particles are created using Discrete Element Methods (DEM) by the legacy solver Grains3D, which randomly determines the centers and angular orientations of cylindrical particles.

The data for this study is derived from my ongoing PhD research at the University of British Columbia. Each simulation ran on the supercomputer Niagara, provided by the Digital Research Alliance of Canada, utilizing 320 cores for 48 hours. The primary constraint of this research is the limited ability to generate new data. The current dataset is used to train a fully connected artificial neural network to predict unseen data.

The simulation parameters include Reynolds number, solid volume fraction, and aspect ratio of the cylinder particles, which respectively quantify the ratio of inertial to viscous forces, the solid particles' volume to the total domain volume, and the cylinder's length to its diameter. Configurations were tested at Reynolds numbers of 5, 10, 20, and 40; solid volume fractions of 0.1, 0.2, and 0.3; and aspect ratios of 1, 2, and 4, resulting in 36 different flow scenarios. To mitigate bias, each scenario was simulated three times with different random particle configurations.

The dataset contains 41,256 cylinders from 108 simulations. Each dataset entry includes the cylinder’s aspect ratio, Reynolds number, and solid volume fraction, followed by 9 scalars representing the particle's rotation matrix, which defines its angular position in space. Subsequent columns provide the positions (x, y, z) relative to the four nearest neighboring particles, adding 12 scalars per row. Additionally, 36 scalars represent the relative rotation matrices for these neighbors, culminating in 60 input parameters per row. The final column indicates the non-dimensional drag force on each particle. A fully connected artificial neural network performs regression analysis to predict the drag force on individual particles.

Importing required packages

In [1]:
import numpy as np
import pandas as pd

!pip uninstall tensorflow -y
!pip install tensorflow==2.12.0

import tensorflow as tf
import warnings

from keras import metrics
from sklearn.metrics import r2_score
from sklearn.model_selection import KFold, cross_val_score, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor

warnings.filterwarnings('ignore')


Found existing installation: tensorflow 2.12.0
Uninstalling tensorflow-2.12.0:
  Successfully uninstalled tensorflow-2.12.0
  Using cached tensorflow-2.12.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (585.9 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-keras 2.15.1 requires tensorflow<2.16,>=2.15, but you have tensorflow 2.12.0 which is incompatible.


In [2]:
tf.__version__

'2.12.0'

Data preprocessing

In [3]:
# Import dataset file
dataset = pd.read_csv('RandomArrayOfCylinders_Drag.csv', header = None)

# Eliminate any bias in the way the dataset was constructed just by shuffling the rows
dataset = dataset.sample(frac = 1).reset_index(drop = True)

dataset

,0,1,2,3,4,5,6,7,8,9,...,51,52,53,54,55,56,57,58,59,60
0,1.0,10.0,0.2,-0.3750,-0.6270,0.683,-0.7100,0.6680,0.2230,-0.59600,...,0.049500,0.657460,0.210388,0.530442,0.194388,0.179694,0.352428,0.053520,0.571290,10.380894
1,4.0,40.0,0.2,-0.2510,-0.3340,-0.908,0.9610,0.0221,-0.2740,0.11200,...,0.091615,-0.792825,0.048384,0.297260,0.009857,-0.093823,0.249700,0.095352,-0.283452,19.035939
2,2.0,40.0,0.1,-0.8460,-0.0442,-0.531,0.0392,-0.9990,0.0206,-0.53200,...,0.283410,-0.009957,0.482524,-0.036553,-0.381618,0.001405,-0.239481,-0.018313,0.069962,5.600875
3,2.0,40.0,0.3,0.7130,0.6710,0.204,0.7010,-0.6710,-0.2430,-0.02660,...,-0.261671,0.650528,0.001673,-0.230153,0.048312,0.296092,0.176460,-0.088695,-0.326112,27.160958
4,2.0,10.0,0.3,-0.5510,0.4350,-0.713,0.8350,0.2770,-0.4760,-0.00938,...,0.147668,0.223780,0.008677,-0.237945,-0.230741,0.070960,0.565409,-0.230384,-0.190550,29.356235
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41251,2.0,40.0,0.1,0.3690,0.2870,-0.884,0.9260,-0.0302,0.3770,0.08150,...,-0.022214,0.913962,0.012225,0.177079,0.002446,-0.748374,-0.693056,0.052403,0.167308,6.322860
41252,2.0,20.0,0.1,-0.2220,0.5170,0.827,-0.7420,0.4610,-0.4870,-0.63300,...,0.046842,0.707126,0.138627,-0.467885,0.048866,-0.297052,-0.305163,-0.138795,-0.249288,8.590264
41253,2.0,40.0,0.3,-0.0311,0.5600,0.828,0.7290,0.5790,-0.3640,-0.68300,...,0.010698,0.574452,-0.348330,0.393680,-0.083376,0.412032,0.515844,-0.217672,0.214704,30.955652
41254,4.0,20.0,0.1,0.9690,-0.1960,0.148,0.1190,0.9030,0.4140,-0.21500,...,-0.910860,0.040460,0.003504,-0.058800,0.727818,0.195330,-0.023680,-0.200790,-0.772280,7.859389


In [4]:
# Creating the matrix of futures and dependent vector
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, -1].values

In [5]:
# Making sure all the input data is non-dimensional, i.e., replacing the relative distances by the non-dimensional relative distances

# Equivalent diameter of each particle
Deq = 1e-3

for i in range(0, X.shape[0]):
  X[i, 12:24] = X[i, 12:24]/Deq

The architecture selected to tackle this problem is a fully connected artificial neural network. Initially, we define the hyperparameters, which include the number of neurons, epochs, batch size, activation functions, learning rate for the Adam optimizer, and the number of k-folds for cross-validation.

The network is constructed using the TensorFlow and Keras libraries and comprises a series of dense, fully connected layers. The architecture begins with an input layer that matches the number of input parameters per row in the dataset, followed by three hidden layers. Each hidden layer utilizes a fixed number of neurons, the 'glorot_uniform' initializer for weight initialization, and a predefined activation function. The output layer features a single neuron with a linear activation function, which is standard for regression tasks.

The Adam optimizer, a method of backpropagation, is employed to train the neural network and adjust the weights. The loss is calculated using the mean squared error (MSE). For model evaluation, the R-squared score is used, implemented through a custom function.

To facilitate cross-validation, a custom KerasRegressor wrapper is utilized along with both normalization and standardization pipelines. K-fold cross-validation is configured to validate the model's performance across various data splits. Finally, a grid search is conducted to optimize the hyperparameters based on their corresponding R-squared scores, ensuring optimal model performance.


ANN model and results

In [6]:
# ANN hyper-parameters
N_neurons = 5 # Number of neurons
N_epochs = 50 # Number of epochs
lr = 0.001 # Learning rate for the Adam optimizer
bs = 191 # Batch size
vb = 0 # Verbose
N_folds = 5 # Number of k-folds
my_seed = 0 # Fixing a seed for the purpose of reproducibility
af_relu = 'relu' # Activation function relu

In [7]:
# R-squared definition function
def R2(y_true, y_pred):

    SS_res = K.sum(K.square(y_true - y_pred))
    SS_tot = K.sum(K.square(y_true - K.mean(y_true)))

    return 1 - SS_res / SS_tot

# Defining the artificial neural network model
def build_model(N_neurons = N_neurons, lr = lr, af = af_relu):

    # Define neural network optmizer
    myoptimizer = tf.keras.optimizers.Adam(learning_rate = lr)

    model = Sequential([
        Dense(units = N_neurons, kernel_initializer = 'glorot_uniform', activation = af, input_shape = (X.shape[1],)),
        Dense(units = N_neurons, kernel_initializer = 'glorot_uniform', activation = af),
        Dense(units = N_neurons, kernel_initializer = 'glorot_uniform', activation = af),
        Dense(units = 1, activation = 'linear', kernel_constraint = tf.keras.constraints.NonNeg())
    ])

    model.compile(optimizer = myoptimizer, loss = 'mse', metrics = [R2])

    return model

# Wrap the model using KerasRegressor
model_wrapper = KerasRegressor(build_fn = build_model, epochs = N_epochs, batch_size = bs, verbose = vb,
                              validation_split = 0.2, shuffle = True)

# Create the pipelines for normalization and standardization respectively
pipeline_normalization = Pipeline([
    ('normalize', MinMaxScaler()),
    ('model', model_wrapper)])

pipeline_standardization = Pipeline([
    ('normalize', StandardScaler()),
    ('model', model_wrapper)])

# Define K-Fold Cross-validation
kf = KFold(n_splits = N_folds, shuffle = True, random_state = my_seed)

# Tuning of hyper-parameters
param_grid = {
    'model__N_neurons': [N_neurons],
    'model__lr': [lr],
    'model__af': [af_relu],
    'model__epochs': [N_epochs],
    'model__batch_size': [bs]
}

# Setup GridSearchCV to find the best hyper-parameters
grid_search_normalization = GridSearchCV(estimator = pipeline_normalization, param_grid = param_grid, cv = kf, scoring = 'r2')
grid_search_standardization = GridSearchCV(estimator = pipeline_standardization, param_grid = param_grid, cv = kf, scoring = 'r2')

# Run the Grid Search
grid_search_normalization.fit(X, y)
grid_search_standardization.fit(X, y)

# Output the best parameters and the corresponding score
print(' ')
print('Normalization: ')
print('Best parameters: ', grid_search_normalization.best_params_)
print('Best R^2 score: ', grid_search_normalization.best_score_)
print(' ')
print('Standardization: ')
print('Best parameters: ', grid_search_standardization.best_params_)
print('Best R^2 score: ', grid_search_standardization.best_score_)


 
Normalization: 
Best parameters:  {'model__N_neurons': 5, 'model__af': 'relu', 'model__batch_size': 191, 'model__epochs': 50, 'model__lr': 0.001}
Best R^2 score:  0.8195829056328877
 
Standardization: 
Best parameters:  {'model__N_neurons': 5, 'model__af': 'relu', 'model__batch_size': 191, 'model__epochs': 50, 'model__lr': 0.001}
Best R^2 score:  0.818491627050846


Results discussion

To reduce computational time, this notebook was implemented with only a fixed set of hyper-parameters in the GridSearch, opting for a relatively small number of neurons and epochs. This configuration required a runtime of approximately 4 minutes and 27 seconds.

Despite the limited scope of hyper-parameter tuning, the results were quite satisfactory. The model displayed no signs of overfitting during the K-fold cross-validation process.

The optimal configuration involved normalizing the data and included an architecture with 5 neurons, a rectified linear unit (ReLU) as the activation function for the dense layers, a batch size of 191, 50 epochs, and a learning rate of 0.001 for the Adam optimizer. This setup achieved the best R-squared score of 0.8195. For further comparison with the available literature, please refer to the specified reference.

Additionally, to explore the model's potential further, I conducted a more extensive GridSearch on my local workstation, equipped with 12 cores, running for 48 hours. The parameters tested were as follows:

param_grid = {
    'model__N_neurons': [10, 20, 40],
    'model__lr': [0.001, 0.01, 0.1],
    'model__af': ['relu', 'tanh', 'sigmoid'],
    'model__epochs': [200, 400, 800],
    'model__batch_size': [191, 382, 573]  
}

The highest performance was achieved using the standardization pipeline, with an architecture comprising 40 neurons, 800 epochs, a sigmoid activation function, a batch size of 191, and a learning rate of 0.01. This configuration yielded an R-squared score of 0.9376. With adequate computational power, the model demonstrates robust performance, significantly surpassing typical R-squared scores reported in the relevant literature (for details, refer to the specified reference, https://doi.org/10.1016/j.ijmultiphaseflow.2023.104603).